In [11]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin

In [17]:
class customlogisticregression(BaseEstimator, ClassifierMixin):
    def __init__(self, learning_rate=0.01, iterations=1000, regularization_strength=0.01):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.regularization_strength = regularization_strength
        self.weights = None
        self.bias = None
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def binary_cross_entropy_loss(self, y_true, y_pred):
        # Avoid log(0) which is undefined
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        
        # Compute binary cross entropy loss
        loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return loss
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # Gradient Descent
        for _ in range(self.iterations):
            # Compute predictions
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(linear_model)

            # Compute loss
            loss = self.binary_cross_entropy_loss(y, y_predicted)
            
            # Compute gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y)) + (self.regularization_strength / n_samples) * self.weights
            db = (1 / n_samples) * np.sum(y_predicted - y)
            
            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            # Print loss
            if _ % 100 == 0:
                print(f"Iteration {_}, Loss: {loss}")
    
    def predict_proba(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        proba = self.sigmoid(linear_model)
        return proba
    
    def predict(self, X):
        return (self.predict_proba(X)>0.5).astype("int")
    
    def score(self, X, y=None):
        predictions = self.predict(X)
        accuracy = np.sum(predictions == y) / len(y)
        return accuracy

In [18]:
# Implementation
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# import dataset
df = pd.read_csv('D:/Advance Machine Learning/heart.csv')
df.head(6)

# drop missing values
df.dropna()

# Define X and y
X = df.drop('target',axis=1)
y = df['target']

# split data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# train model with modification parameter
model = customlogisticregression()
model.fit(X_train,y_train)

# predict model
y_pred = model.predict(X_test)

# show accuracy
print(accuracy_score(y_test,y_pred))


Iteration 0, Loss: 0.6931471805599453


C:\Users\jkurn\AppData\Local\Temp\ipykernel_20256\2760480069.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Iteration 100, Loss: 12.82245531126914
Iteration 200, Loss: 14.619638395961367
Iteration 300, Loss: 13.878662351560857
Iteration 400, Loss: 13.85879440075982
Iteration 500, Loss: 13.84162965561909
Iteration 600, Loss: 13.83137710015052
Iteration 700, Loss: 13.806580485316395
Iteration 800, Loss: 13.71403796906074
Iteration 900, Loss: 13.662220364368569
0.639344262295082


In [19]:
## Hyperparameter TUning
param_grid = {
    'learning_rate': [0.001, 0.01],
    'iterations': [1000, 2000 ]
}

# RandomizedSearchCV for hyperparameter tuning
randomized_cv = RandomizedSearchCV(model, param_distributions=param_grid, cv=3, n_iter=5)
randomized_cv.fit(X_train, y_train)

print("Best hyperparameters (RandomizedSearchCV):", randomized_cv.best_params_)
print("Best score (RandomizedSearchCV):", randomized_cv.best_score_)


# GridSearchCV for hyperparameter tuning
grid_cv = GridSearchCV(model, param_grid=param_grid, cv=3)
grid_cv.fit(X_train, y_train)

print("Best hyperparameters (GridSearchCV):", grid_cv.best_params_)
print("Best score (GridSearchCV):", grid_cv.best_score_)

# Predict using the best model from GridSearchCV
best_lr_model = grid_cv.best_estimator_
predictions = best_lr_model.predict(X_test)

# Evaluate the model
accuracy = np.sum(predictions == y_test) / len(y_test)
print("Accuracy:", accuracy)

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=5. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Iteration 0, Loss: 0.6931471805599453
Iteration 100, Loss: 12.515966832728044
Iteration 200, Loss: 10.764784566239936
Iteration 300, Loss: 8.934932413898208
Iteration 400, Loss: 8.684803019761922
Iteration 500, Loss: 8.611695412749906
Iteration 600, Loss: 8.586760185343984
Iteration 700, Loss: 8.569804788356231
Iteration 800, Loss: 8.554401459381353
Iteration 900, Loss: 8.53642635156368
Iteration 0, Loss: 0.6931471805599453
Iteration 100, Loss: 12.219039364484878
Iteration 200, Loss: 9.533236841446046
Iteration 300, Loss: 8.6040428141707
Iteration 400, Loss: 8.280166032389689
Iteration 500, Loss: 8.145728638536447
Iteration 600, Loss: 8.040953202361207
Iteration 700, Loss: 7.962610324832358
Iteration 800, Loss: 7.921574908888183
Iteration 900, Loss: 7.90425020323281
Iteration 0, Loss: 0.6931471805599453
Iteration 100, Loss: 13.260124045835934
Iteration 200, Loss: 9.156394032149322
Iteration 300, Loss: 8.509407328838902
Iteration 400, Loss: 8.451962106167496
Iteration 500, Loss: 8.49668

C:\Users\jkurn\AppData\Local\Temp\ipykernel_20256\2760480069.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Iteration 100, Loss: 15.660801536899903
Iteration 200, Loss: 15.334040448676737
Iteration 300, Loss: 13.562088051151285
Iteration 400, Loss: 13.329374973785844
Iteration 500, Loss: 13.221619431988671
Iteration 600, Loss: 13.227892988315842
Iteration 700, Loss: 13.241943213258706
Iteration 800, Loss: 13.250950911586244
Iteration 900, Loss: 13.257493316656138
Iteration 0, Loss: 0.6931471805599453
Iteration 100, Loss: 15.446270008997162
Iteration 200, Loss: 14.080702735968416
Iteration 300, Loss: 13.56359617373094
Iteration 400, Loss: 13.056251272176342
Iteration 500, Loss: 13.243307015585708
Iteration 600, Loss: 13.204774219119324
Iteration 700, Loss: 13.134972876280111
Iteration 800, Loss: 13.088317754061661
Iteration 900, Loss: 13.088786287253438
Iteration 0, Loss: 0.6931471805599453
Iteration 100, Loss: 15.564129922474592
Iteration 200, Loss: 13.984133870345701
Iteration 300, Loss: 13.34775432756371
Iteration 400, Loss: 13.288573384383175
Iteration 500, Loss: 13.323275363207964
Iterat

C:\Users\jkurn\AppData\Local\Temp\ipykernel_20256\2760480069.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Iteration 100, Loss: 15.660801536899903
Iteration 200, Loss: 15.334040448676737
Iteration 300, Loss: 13.562088051151285
Iteration 400, Loss: 13.329374973785844
Iteration 500, Loss: 13.221619431988671
Iteration 600, Loss: 13.227892988315842
Iteration 700, Loss: 13.241943213258706
Iteration 800, Loss: 13.250950911586244
Iteration 900, Loss: 13.257493316656138
Iteration 0, Loss: 0.6931471805599453
Iteration 100, Loss: 15.446270008997162
Iteration 200, Loss: 14.080702735968416
Iteration 300, Loss: 13.56359617373094
Iteration 400, Loss: 13.056251272176342
Iteration 500, Loss: 13.243307015585708
Iteration 600, Loss: 13.204774219119324
Iteration 700, Loss: 13.134972876280111
Iteration 800, Loss: 13.088317754061661
Iteration 900, Loss: 13.088786287253438
Iteration 0, Loss: 0.6931471805599453
Iteration 100, Loss: 15.564129922474592
Iteration 200, Loss: 13.984133870345701
Iteration 300, Loss: 13.34775432756371
Iteration 400, Loss: 13.288573384383175
Iteration 500, Loss: 13.323275363207964
Iterat